In [ ]:
import torch
import pandas as pd
from torchvision.io import read_image
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import os
from PIL import Image
from skimage import io
import numpy as np
from torch import nn
import torchvision.models as models
from torch.utils.data import DataLoader
import torch.optim as optim



In [ ]:
class CustomImageDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.data = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir,
                        str(self.data.iloc[idx, 0]))
        
        image = io.imread(img_path)
        label = torch.tensor(int(self.data.iloc[idx, 1]))
        
        if self.transform:
            image = self.transform(image)
        
        return (image, label)

In [ ]:
import torchvision.transforms as transforms

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((96, 96), antialias=False),
])


training_data = CustomImageDataset("the-flintstones-face-detection2/train_labels.csv", "the-flintstones-face-detection2/data", transform=transform)
validation_data = CustomImageDataset("the-flintstones-face-detection2/validation_labels.csv", "the-flintstones-face-detection2/data", transform=transform)

In [ ]:
train_dataloader = DataLoader(training_data, batch_size = 64, shuffle = True)
validation_dataloader = DataLoader(validation_data, batch_size = 64, shuffle = True)


In [ ]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

In [ ]:
resnet18 = models.resnet18()

In [ ]:
resnet18.fc = nn.Sequential(
    nn.Linear(resnet18.fc.in_features, 2),
    nn.Sigmoid()
)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet18.parameters(), lr=1e-3)

num_epochs = 10
losses = []

resnet18.to(device)

for epoch in range(num_epochs):
    for inputs, labels in train_dataloader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = resnet18(inputs)
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')
    losses.append(loss.item())

In [ ]:
results = []

def acc(loader, model, num_samples):
    correct = 0
    model.eval()

    with torch.no_grad():

        for inputs, labels in loader:
            # incarcam datele pe gpu
            inputs = inputs.to(device)
            labels = labels.to(device)

            # facem predictiile
            scores = model(inputs)
            _, prd = scores.max(1)

            results.extend(prd.cpu().tolist())

            correct += (prd == labels).sum()
        
        print(f'acc = {correct / num_samples:.7f}')

        
print(acc(train_dataloader, resnet18, len(training_data)))
print(acc(validation_dataloader, resnet18, len(validation_data)))


In [ ]:
torch.save(resnet18,'/kaggle/working/resnet18_trained_weights.pth')